# <center> Algorithmic Economics </center>
## <center> Assignment 2 </center>

The due date for this assignment is __Wednesday, February 28th by 9am EST__. To complete this assignment, fill out the notebook with your answers and executed python code, and then submit the filled out notebook. The solution should be submitted via Microsoft Teams as a file attachment.

You are in charge of network optimization for a local branch of a global delivery company with 22 local sort centers. The company is renewing the fleet of its delivery trucks and you have been tasked with determining the allocation of trucks to each of the 22 sort centers. The head office of the company told you that the local branch will be allocated as many trucks as needed as long as you are able to to prove that they are necessary for operations. You were given the following parameters:

* Each truck is capable of carrying 100,000 lbs. of freight per day and has a daily operating cost of $2,500 (including driver’s salary). Note that it is possible to load the truck with less than 100,000 lbs. of freight but that will not change its operating cost.

* For any freight that your trucks will not be able to deliver, the head office has contracted with a small local delivery company wich promises to deliver any (as little or as much) extra freight at the price of 10 cents per pound from any sort center.

To make a data-driven request for truck allocation to the head office, you have tasked your data scientist to analyze daily demand for freight shipments from local customers at each of the 22 sort centers. The demand distribution in each center is very close to normal. Your data scientist collected means and standard deviations of the daily demand at each sort center (expressed in million lbs. of freight) in the attached file *"freight\_demand\_estimates"*.

1. __(10 points)__ Suppose that D is the demand for freight (in pounds) delivered from a given sort center and N is the number of trucks it has. Provided the cost figures above, construct the cost function which will give you the cost in $ for operating freight delivery from that sort center with N trucks as a function of the freight demand D at that sort center.

__Solution__: Construct the cost function.

$$
cost(N,D) = 2500N + max(0, (D - 100000N)) \times 0.10
$$

In [1]:
def cost(N, D):
    capacity = N * 100000
    cost = 2500 * N + max(0, D - capacity) * 0.10
    return cost


2. __(20 points)__ Assume that there is no demand uncertainty and the mean demand figures reported in *“freight\_demand\_estimates”* are the actual demand realizations in each of the sort centers. Using the cost function you constructed in the previous question, find the number of trucks you need to allocate to each sort center to minimize the operating cost of the branch.

__Solution__

In [2]:
# !pip install --user pandas openpyxl

In [3]:
# Load the data
import pandas as pd

freight_demand_est = pd.read_excel('freight_demand_estimates.xlsx')
freight_demand_est

# Process the data

,Sort center #,Daily demand (in million lbs),Standard deviation of demand (in million lbs)
0,1,1.291199,2.776397
1,2,7.703011,2.723797
2,3,9.106282,0.007215
3,4,21.946484,3.885373
4,5,8.091064,1.848259
5,6,5.595920,1.463182
6,7,6.595868,0.585117
7,8,13.524867,3.168778
8,9,0.341742,1.728683
9,10,9.399122,0.408743


In [4]:
def get_optimal_number_of_trucks(demand): # demand in lbs
    '''
        Return the optimal number of trucks for a sort center if there is no demand uncertainty.
        Parameters:
            demand: Exact demand at the sort center.
    '''
    ntrucks = int(demand // 100000 ) # number of trucks we can fill
    if cost(ntrucks + 1, demand) < cost(ntrucks, demand): #if the cost of using one more truck is cheaper, use another truck
        ntrucks += 1

    print(ntrucks)
    return ntrucks

In [5]:
for _, (sort_center, demand_mu, _) in freight_demand_est.iterrows():
    get_optimal_number_of_trucks(demand_mu * 1000000) # demand_mu in million lbs so multiply by 1000000

13
77
91
220
81
56
66
135
4
94
30
133
147
31
31
49
185
129
141
54
32
4


3. __(30 points)__ Now suppose that the demand for freight (in pounds) in a sort center is random day-to-day and normally distributed with mean $\mu$ and standard deviation $\sigma$. Provided the cost figures above, construct the expected cost function which will give you the expected daily cost in $ for operating freight delivery from that sort center as a function of the number of trucks N allocated to that sort center.

__Solution__: Construct the expected cost function.

$$
cost(N,D) = 2500N + max(0, (D - 100000N)) \times 0.10
$$

$$
E[cost(N,D)] =  2500N + E[max(0, (D - 100000N)) \times 0.10]
$$

N is given, but D is a normal random variable with mean $\mu$ and standard deviation $\sigma$

In [6]:
from scipy.integrate import quad
from scipy.stats import norm
import numpy as np



In [7]:
def get_expected_cost(number_of_trucks, demand_mu, demand_sigma):
    '''
        Return the expected cost of a sort center.
        Parameters:
            number_of_trucks: Number of trucks assigned to the sort center.
            demand_mu: Mean demand at the sort center
            demand_sigma: Standard deviation of demand at the sort center
    '''
    capacity = 100000 * number_of_trucks
    cost = 2500 * number_of_trucks

    # expected additional cost when demand exceeds capacity
    additional_cost = lambda x: x - capacity

    # Calculate the expected additional cost using norm.expect
    expected_additional_cost = norm.expect(additional_cost, loc=demand_mu, scale=demand_sigma, lb=capacity)

    total_expected_cost = cost + max(0,expected_additional_cost) * 0.10

    return total_expected_cost
    

4. __(30 points)__ Using the numbers reported in *"freight\_demand\_estimates"* for mean and standard deviation of the demand for freight in each of the 22 sort centers, find the number of trucks N which needs to be allocated to each sort center to minimize the expected cost.

__Solution__:

In [8]:
from scipy.optimize import minimize_scalar

In [9]:
# For each sort center, find the number of trucks that minimizes the expected cost at that sort center

optimal_sort_center_trucks = []
optimal_costs = []

for i, (sort_center, demand_mu, demand_sigma) in freight_demand_est.iterrows():
    demand_mu_lbs = demand_mu * 1000000
    demand_sigma_lbs = demand_sigma * 1000000 # covert from millions of lbs to lbs
    min_trucks = 5
    min_cost = get_expected_cost(min_trucks, demand_mu_lbs, demand_sigma_lbs)

    for i in range(5,300):
        cost = get_expected_cost(i, demand_mu_lbs, demand_sigma_lbs)

        if cost < min_cost:
            min_trucks = i
            min_cost = cost
    
    optimal_sort_center_trucks.append(min_trucks)
    optimal_costs.append(min_cost)
    print(f"Sort Center: {sort_center}, Demand (mu): {demand_mu_lbs}, Demand (sigma): {demand_sigma_lbs}, Optimal Trucks: {min_trucks}, Optimal Cost: ${min_cost:,.2f}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2914: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  dub = integrate.quad(fun, d, ub, **kwds)[0]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2894: RuntimeWarning: invalid value encountered in scalar multiply
  return func(x) * self.pdf(x, *args, **lockwds)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2914: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  dub = integrate.quad(fun, d, ub, **kwds)[0]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2913: IntegrationWarning: The occurrence of roundoff error is detect

Sort Center: 1.0, Demand (mu): 1291198.8378673345, Demand (sigma): 2776396.607402174, Optimal Trucks: 28, Optimal Cost: $113,184.64
Sort Center: 2.0, Demand (mu): 7703011.068146806, Demand (sigma): 2723796.6748050964, Optimal Trucks: 92, Optimal Cost: $271,947.73
Sort Center: 3.0, Demand (mu): 9106281.970117815, Demand (sigma): 7215.204028525157, Optimal Trucks: 91, Optimal Cost: $228,204.64
Sort Center: 4.0, Demand (mu): 21946483.8983367, Demand (sigma): 3885373.14902582, Optimal Trucks: 241, Optimal Cost: $661,886.89
Sort Center: 5.0, Demand (mu): 8091063.533493584, Demand (sigma): 1848258.7617615035, Optimal Trucks: 91, Optimal Cost: $256,135.06


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2914: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  dub = integrate.quad(fun, d, ub, **kwds)[0]


Sort Center: 6.0, Demand (mu): 5595919.739296118, Demand (sigma): 1463181.6151159792, Optimal Trucks: 64, Optimal Cost: $182,535.64
Sort Center: 7.0, Demand (mu): 6595867.614291451, Demand (sigma): 585116.8785533218, Optimal Trucks: 69, Optimal Cost: $181,952.90
Sort Center: 8.0, Demand (mu): 13524866.9618148, Demand (sigma): 3168778.144400176, Optimal Trucks: 153, Optimal Cost: $430,470.64
Sort Center: 9.0, Demand (mu): 341742.37092552404, Demand (sigma): 1728682.6293116037, Optimal Trucks: 12, Optimal Cost: $58,980.71
Sort Center: 10.0, Demand (mu): 9399121.679456102, Demand (sigma): 408743.3260170044, Optimal Trucks: 97, Optimal Cost: $247,991.26
Sort Center: 11.0, Demand (mu): 2947873.982137356, Demand (sigma): 641325.5880032791, Optimal Trucks: 33, Optimal Cost: $92,385.27
Sort Center: 12.0, Demand (mu): 13233265.4855224, Demand (sigma): 3505712.871994191, Optimal Trucks: 151, Optimal Cost: $432,997.39
Sort Center: 13.0, Demand (mu): 14663682.491229301, Demand (sigma): 3160427.370

In [10]:
optimal_sort_center_trucks

[28,
 92,
 91,
 241,
 91,
 64,
 69,
 153,
 12,
 97,
 33,
 151,
 164,
 34,
 44,
 65,
 207,
 150,
 162,
 59,
 53,
 14]

5. (10 points) What is the total expected daily cost of optimal operation of the freight delivery branch you are in charge of?

In [11]:
# Calculate total expected daily cost of optimal operation of the freight delivery branch
total_expected_daily_cost = sum(optimal_costs)
round(total_expected_daily_cost,2)

5962121.22

Total expected daily cost is $5,962,060.14